In [1]:
!pip install google-cloud-vision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 6.7 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()


Saving prescription.jpg to prescription.jpg


In [7]:
import base64
import requests

# 📌 Your OpenAI API key
api_key = "OPEN_AI_API_KEY"  # Replace with your actual GPT-4 Vision-capable key

# 📷 Load and encode image to base64
image_path = "prescription.jpg"  # or your processed image
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

# 🔧 GPT-4 Vision API endpoint
url = "https://api.openai.com/v1/chat/completions"

# 🧾 Prompt and message setup
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4o-mini", # Updated model name to a current GPT-4 Vision model
    "messages": [
        {
            "role": "system",
            "content": (
                "You are a medical assistant. A user is uploading a handwritten prescription. "
                "Your job is to extract structured details from it including:\n"
                "- Patient name, age, sex, address\n"
                "- Date of prescription\n"
                "- List of medications: name, dosage, quantity, instructions\n"
                "- Physician’s name, license number, PTR number, and signature if visible\n"
                "Be accurate and clearly mark any uncertain or unreadable parts."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Please extract the structured information from this handwritten prescription."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 1000
}

# 🧠 Make the request to GPT-4 Vision
response = requests.post(url, headers=headers, json=payload)

# 📤 Print the structured output
result = response.json()
print("\n📑 Structured Prescription Output:\n")
print(result["choices"][0]["message"]["content"])


📑 Structured Prescription Output:

Here’s the structured information extracted from the handwritten prescription:

- **Patient Information:**
  - Name: Libita Alvarez
  - Address: Bagong B, Pasig City
  - Age: 39
  - Sex: F

- **Date of Prescription:** 2-5-13

- **List of Medications:**
  1. **FeSO4 tab**
     - Quantity: #30
     - Instructions (Sig): A.D.
  2. **Ascorbic Acid**
     - Quantity: #30 (tablets)
     - Instructions (Sig): Once a day

- **Physician Information:**
  - Physician’s Name: [Uncertain, appears as "Jolacan" or "Jelacan"]
  - License Number: 12345
  - PTR Number: 123467
  - Signature: [Visible but unclear]

If you need any further details or clarifications, feel free to ask!


**Streamlit**

In [8]:
# Install dependencies
!pip install streamlit pyngrok --quiet

# Save the Streamlit app to app.py
app_code = """
import streamlit as st
import base64
import requests
from PIL import Image
import io

st.set_page_config(page_title="Prescription Extractor", layout="wide")
st.title("📋 Handwritten Prescription Extractor")
st.markdown("Upload a prescription image to extract structured patient and medication details .")

uploaded_file = st.file_uploader("📎 Upload a Prescription Image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")  # ensure compatibility
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    base64_image = base64.b64encode(buffered.getvalue()).decode()

    api_key = st.text_input("🔐 Enter your OpenAI API Key", type="password")

    if st.button("🧠 Extract Information") and api_key:
        with st.spinner("Calling GPT-4 Vision API..."):
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            }
            payload = {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": (
                            "You are a medical assistant. A user is uploading a handwritten prescription. "
                            "Your job is to extract structured details from it including:\\n"
                            "- Patient name, age, sex, address\\n"
                            "- Date of prescription\\n"
                            "- Medical reason for the prescription (diagnosis, symptoms, or complaint)\\n"
                            "- List of medications: name, dosage, quantity, instructions\\n"
                            "- Physician’s name, license number, PTR number, and signature if visible\\n"
                            "Be accurate and clearly mark any uncertain or unreadable parts."
                        )
                    },
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "Please extract the structured information from this handwritten prescription."},
                            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                        ]
                    }
                ],
                "max_tokens": 1000
            }

            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

            col1, col2 = st.columns(2)
            with col1:
                st.image(image, caption="🖼️ Uploaded Prescription", use_column_width=True)
            with col2:
                if response.status_code == 200:
                    result = response.json()["choices"][0]["message"]["content"]
                    st.success("✅ Extraction Complete")
                    st.markdown("### 📑 Structured Output")
                    st.markdown(result)
                else:
                    st.error(f"❌ API Error: {response.status_code}")
                    st.text(response.text)
"""

with open("app.py", "w") as f:
    f.write(app_code)

# Run Streamlit with ngrok
from pyngrok import ngrok
import time, os

NGROK_AUTH_TOKEN = "MGROK_API_FROM THE WEBSITE"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()

# Run Streamlit app
os.system("streamlit run app.py &")
time.sleep(5)

# Get public URL
public_url = ngrok.connect(8501)
print("🚀 Your app is live at:", public_url.public_url)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


ERROR:pyngrok.process.ngrok:t=2025-08-04T20:35:22+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-04T20:35:22+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
!pip install pyngrok